<a href="https://colab.research.google.com/github/isakimfv/TutorialesPython/blob/main/Como_usar_Open_Meteo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Librerías para la realización de solicitudes a la API
!pip install openmeteo_requests
!pip install requests_cache
!pip install retry_requests

#Librerías para el manejo de los datos
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.4 MB/s eta 0:00:00


In [4]:
#Librerías para la realización de solicitudes a la API
import openmeteo_requests
import requests_cache
from retry_requests import retry     #solo es necesario el método retry de esta librería

#Librerías para el manejo de los datos
import pandas as pd                           #Se identificará pandas como pd y
import numpy as np                           #numpy como np para simplificar el código.


In [5]:
#Se establece el identificador de la cache (.cache) y su duración (60 segundos)
cache_session = requests_cache.CachedSession('.cache', expire_after = 60)

#Se define los retry indicando la caché, la cantidad de intentos (5) y el factor del tiempo de espera
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)

#Finalmente, se crea el objeto de la solicitud con todo lo desarrollado
openmeteo = openmeteo_requests.Client(session = retry_session)

In [8]:
#El URL del tipo de dato que se desea, actuales, pronósticos o históricos (Como es el caso).
url = "https://archive-api.open-meteo.com/v1/archive"

#Parámetros de los datos deseados en forma de diccionario.
params = {
	"latitude": 10.488,                              #Latitud de la región
	"longitude": -66.8792,                       #Longitud de la región
	"start_date": "2024-07-19",                #Fecha de inicio de los datos históricos
	"end_date": "2024-08-02",                 #Fecha de fin

                 #Variables meteorológicas en forma horaria (Así se solicita más de una)
	"hourly": ["temperature_2m", "wind_speed_10m"],

                #Variables meteorológicas en formato diario (Así se solicita una sola)
                "daily": "precipitation_sum",
                "timezone": "auto"
}

#Se indican los parámetros a la solicitud, se ejecuta y se asigna los resultados la objeto responses
responses = openmeteo.weather_api(url, params=params)

In [9]:
response = responses[0] #Primera respuesta del arreglo de respuesta

#Parámetros de la primera respuesta
print(f"Coordenadas {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevación {response.Elevation()} m asl")
print(f"Zona Temporaria {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Diferencia con GMT+0 {response.UtcOffsetSeconds()} s")

Coordenadas 10.509666442871094°N -66.92510986328125°E
Elevación 894.0 m asl
Zona Temporaria b'Etc/GMT+4' b'-04'
Diferencia con GMT+0 -14400 s


In [10]:
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()

In [11]:
#Creación del índice date con el inicio, final e intervalo correcto.
hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
#Se adjuntan los valores meteorológicos
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

#Se crea el Dataframe
hourly_dataframe = pd.DataFrame(data = hourly_data)

#Se muestran los primeros valores del Dataframe
hourly_dataframe.head()

,date,temperature_2m,wind_speed_10m
0,2024-07-19 04:00:00+00:00,20.443001,5.447788
1,2024-07-19 05:00:00+00:00,20.093000,4.334974
2,2024-07-19 06:00:00+00:00,19.993000,5.506941
3,2024-07-19 07:00:00+00:00,19.743000,6.214563
4,2024-07-19 08:00:00+00:00,19.243000,6.214563


In [12]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["precipitation_sum"] = daily_precipitation_sum

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe.head()

,date,precipitation_sum
0,2024-07-19 04:00:00+00:00,2.6
1,2024-07-20 04:00:00+00:00,1.4
2,2024-07-21 04:00:00+00:00,1.3
3,2024-07-22 04:00:00+00:00,2.3
4,2024-07-23 04:00:00+00:00,3.1
